In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

df = pd.read_csv('fruit_veg_fresh_stale.csv')

df = df[['image_path', 'type']]  

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['type'], random_state=42)

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='type',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='type',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
out = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint_cb = ModelCheckpoint(
    "fruit_vs_vegetable_checkpoint.h5",
    save_weights_only=False,     
    save_best_only=False,
    save_freq='epoch',
    verbose=1
)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint_cb]
)

model.save("fruit_vs_vegetable_model.h5")


Found 44658 validated image filenames belonging to 2 classes.
Found 11165 validated image filenames belonging to 2 classes.
Epoch 1/10
 450/1396 [========>.....................] - ETA: 8:33 - loss: 0.2051 - accuracy: 0.9292

c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1396/1396 [==============================] - ETA: 0s - loss: 0.1257 - accuracy: 0.9582
Epoch 1: saving model to fruit_vs_vegetable_checkpoint.h5


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1396/1396 [==============================] - 1030s 735ms/step - loss: 0.1257 - accuracy: 0.9582 - val_loss: 0.0664 - val_accuracy: 0.9807
Epoch 2/10
1396/1396 [==============================] - ETA: 0s - loss: 0.0523 - accuracy: 0.9843
Epoch 2: saving model to fruit_vs_vegetable_checkpoint.h5
1396/1396 [==============================] - 1348s 966ms/step - loss: 0.0523 - accuracy: 0.9843 - val_loss: 0.0471 - val_accuracy: 0.9863
Epoch 3/10
1396/1396 [==============================] - ETA: 0s - loss: 0.0350 - accuracy: 0.9897
Epoch 3: saving model to fruit_vs_vegetable_checkpoint.h5
1396/1396 [==============================] - 1846s 1s/step - loss: 0.0350 - accuracy: 0.9897 - val_loss: 0.0367 - val_accuracy: 0.9893
Epoch 4/10
1396/1396 [==============================] - ETA: 0s - loss: 0.0251 - accuracy: 0.9929
Epoch 4: saving model to fruit_vs_vegetable_checkpoint.h5
1396/1396 [==============================] - 1090s 781ms/step - loss: 0.0251 - accuracy: 0.9929 - val_loss: 0.0305 - val_

KeyboardInterrupt: 

In [3]:
from tensorflow.keras.models import load_model

model = load_model("fruit_vs_vegetable_checkpoint.h5")

# Resume training from epoch 3
model.fit(
    train_gen,
    validation_data=val_gen,
    initial_epoch=5,
    epochs=10,
    callbacks=[checkpoint_cb]
)

Epoch 6/10
 729/1396 [==============>...............] - ETA: 6:12 - loss: 0.0186 - accuracy: 0.9949

c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1396/1396 [==============================] - ETA: 0s - loss: 0.0187 - accuracy: 0.9947
Epoch 6: saving model to fruit_vs_vegetable_checkpoint.h5


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1396/1396 [==============================] - 986s 704ms/step - loss: 0.0187 - accuracy: 0.9947 - val_loss: 0.0295 - val_accuracy: 0.9912
Epoch 7/10
1396/1396 [==============================] - ETA: 0s - loss: 0.0140 - accuracy: 0.9963
Epoch 7: saving model to fruit_vs_vegetable_checkpoint.h5
1396/1396 [==============================] - 1006s 721ms/step - loss: 0.0140 - accuracy: 0.9963 - val_loss: 0.0271 - val_accuracy: 0.9916
Epoch 8/10
  25/1396 [..............................] - ETA: 13:46 - loss: 0.0157 - accuracy: 0.9962

KeyboardInterrupt: 

In [4]:
model.save("fruit_vs_vegetable_model.keras")


In [1]:
from tensorflow.keras.models import load_model

model = load_model("fruit_vs_vegetable_model.keras")
model.save("fruit_vs_vegetable_model.h5")


c:\Users\Heena\.conda\envs\btp_venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
